<a href="https://colab.research.google.com/github/groda/big_data/blob/master/MapReduce_Primer_HelloWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/groda/big_data"><div><img src="https://github.com/groda/big_data/blob/master/logo_bdb.png?raw=true" align=right width="90"></div></a>

# MapReduce: A Primer with <code>Hello World!</code>
<br>
<br>

For this tutorial, we are going to download the core Hadoop distribution and run Hadoop in _local standalone mode_:

> ❝ _By default, Hadoop is configured to run in a non-distributed mode, as a single Java process._ ❞

(see [https://hadoop.apache.org/docs/stable/.../Standalone_Operation](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html#Standalone_Operation))

We are going to run a MapReduce job using MapReduce's [streaming application](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Hadoop_Streaming). This is not to be confused with real-time streaming:

> ❝ _Hadoop streaming is a utility that comes with the Hadoop distribution. The utility allows you to create and run Map/Reduce jobs with any executable or script as the mapper and/or the reducer._ ❞

MapReduce streaming defaults to using `IdentityMapper` `IdentityReducer`, thus eliminating the need for explicit specification of a mapper or reducer. Finally, we show how to run a map-only job by setting `mapreduce.job.reduce` equal to $0$.

# Download core Hadoop



In [1]:
HADOOP_URL = "https://dlcdn.apache.org/hadoop/common/stable/hadoop-3.3.6.tar.gz"

import requests
import os
import tarfile

def download_and_extract_targz(url):
    response = requests.get(url)
    filename = url.rsplit('/', 1)[-1]
    HADOOP_HOME = filename[:-7]
    # set HADOOP_HOME environment variable
    os.environ['HADOOP_HOME'] = HADOOP_HOME
    if os.path.isdir(HADOOP_HOME):
      print("Not downloading, Hadoop folder {} already exists".format(HADOOP_HOME))
      return
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        with tarfile.open(filename, 'r:gz') as tar_ref:
            extract_path = tar_ref.extractall(path='.')
            # Get the names of all members (files and directories) in the archive
            all_members = tar_ref.getnames()
            # If there is a top-level directory, get its name
            if all_members:
              top_level_directory = all_members[0]
              print(f"ZIP file downloaded and extracted successfully. Contents saved at: {top_level_directory}")
    else:
        print(f"Failed to download ZIP file. Status code: {response.status_code}")


download_and_extract_targz(HADOOP_URL)

ZIP file downloaded and extracted successfully. Contents saved at: hadoop-3.3.6


# Set environment variables

## Set `HADOOP_HOME` and `PATH`

In [2]:
# HADOOP_HOME was set earlier when downloading Hadoop distribution
print("HADOOP_HOME is {}".format(os.environ['HADOOP_HOME']))

os.environ['PATH'] = ':'.join([os.path.join(os.environ['HADOOP_HOME'], 'bin'), os.environ['PATH']])
print("PATH is {}".format(os.environ['PATH']))

HADOOP_HOME is hadoop-3.3.6
PATH is hadoop-3.3.6/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


## Set `JAVA_HOME`

While Java is readily available on Google Colab, we consider the broader scenario of an Ubuntu machine. In this case, we ensure compatibility by installing Java, specifically opting for the `openjdk-19-jre-headless` version.

In [3]:
import shutil

# set variable JAVA_HOME (install Java if necessary)
def is_java_installed():
    os.environ['JAVA_HOME'] = os.path.realpath(shutil.which("java")).split('/bin')[0]
    return os.environ['JAVA_HOME']

def install_java():
    # Uncomment and modify the desired version
    # java_version= 'openjdk-11-jre-headless'
    # java_version= 'default-jre'
    # java_version= 'openjdk-17-jre-headless'
    # java_version= 'openjdk-18-jre-headless'
    java_version= 'openjdk-19-jre-headless'

    print(f"Java not found. Installing {java_version} ... (this might take a while)")
    try:
        cmd = f"apt install -y {java_version}"
        subprocess_output = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        stdout_result = subprocess_output.stdout
        # Process the results as needed
        print("Done installing Java {}".format(java_version))
        os.environ['JAVA_HOME'] = os.path.realpath(shutil.which("java")).split('/bin')[0]
        print("JAVA_HOME is {}".format(os.environ['JAVA_HOME']))
    except subprocess.CalledProcessError as e:
        # Handle the error if the command returns a non-zero exit code
        print("Command failed with return code {}".format(e.returncode))
        print("stdout: {}".format(e.stdout))

# Install Java if not available
if is_java_installed():
    print("Java is already installed: {}".format(os.environ['JAVA_HOME']))
else:
    print("Installing Java")
    install_java()

Java is already installed: /usr/lib/jvm/java-11-openjdk-amd64


# Run a MapReduce job with Hadoop streaming

## Create a file

Write the string"Hello, World!" to a local file.<p>**Note:** you will be writing to the file `./hello.txt` in your current directory (denoted by `./`).

In [4]:
!echo "Hello, World!">./hello.txt

## Launch the MapReduce "Hello, World!" application

Since the default filesystem is the local filesystem (as opposed to HDFS) we do not need to upload the local file `hello.txt` to HDFS.

Run a MapReduce job with `/bin/cat` as a mapper and no reducer.

**Note:** the first step of removing the output directory is necessary because MapReduce does not overwrite data folders bt design.

In [5]:
%%bash
hdfs dfs -rm -r my_output

mapred streaming \
    -input hello.txt \
    -output my_output \
    -mapper '/bin/cat'

rm: `my_output': No such file or directory
2024-03-09 19:25:25,923 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-03-09 19:25:26,187 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-03-09 19:25:26,187 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-03-09 19:25:26,217 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-03-09 19:25:26,555 INFO mapred.FileInputFormat: Total input files to process : 1
2024-03-09 19:25:26,597 INFO mapreduce.JobSubmitter: number of splits:1
2024-03-09 19:25:27,032 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local451480222_0001
2024-03-09 19:25:27,033 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-03-09 19:25:27,317 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2024-03-09 19:25:27,319 INFO mapreduce.Job: Running job: job_local451480222_0001
2024-03-09 19:25:27,332 INFO mapred.LocalJobRunner: O

## Verify the result

If the job executed successfully, an empty file named `_SUCCESS` is expected to be present in the output directory `my_output`.

Verify the success of the MapReduce job by checking for the presence of the `_SUCCESS` file.

In [6]:
%%bash

echo "Check if MapReduce job was successful"
hdfs dfs -test -e my_output/_SUCCESS
if [ $? -eq 0 ]; then
	echo "_SUCCESS exists!"
fi

Check if MapReduce job was successful
_SUCCESS exists!


**Note:** `hdfs dfs -ls` is the same as `ls` since the default filesystem is the local filesystem.

In [7]:
!hdfs dfs -ls my_output

Found 2 items
-rw-r--r--   1 root root          0 2024-03-09 19:25 my_output/_SUCCESS
-rw-r--r--   1 root root         15 2024-03-09 19:25 my_output/part-00000


In [8]:
!ls -l my_output

total 4
-rw-r--r-- 1 root root 15 Mar  9 19:25 part-00000
-rw-r--r-- 1 root root  0 Mar  9 19:25 _SUCCESS


The actual output of the MapReduce job is contained in the file `part-00000` in the output directory.

In [9]:
!cat my_output/part-00000

Hello, World!	


# MapReduce without specifying mapper or reducer

In the previous example, we have seen how to run a MapReduce job without specifying any reducer.

Since the only required options for `mapred streaming` are `input` and `output`, we can also run a MapReduce job without specifying a mapper.

In [10]:
!mapred streaming -h

2024-03-09 19:25:33,805 ERROR streaming.StreamJob: Unrecognized option: -h
Usage: $HADOOP_HOME/bin/hadoop jar hadoop-streaming.jar [options]
Options:
  -input          <path> DFS input file(s) for the Map step.
  -output         <path> DFS output directory for the Reduce step.
  -mapper         <cmd|JavaClassName> Optional. Command to be run as mapper.
  -combiner       <cmd|JavaClassName> Optional. Command to be run as combiner.
  -reducer        <cmd|JavaClassName> Optional. Command to be run as reducer.
  -file           <file> Optional. File/dir to be shipped in the Job jar file.
                  Deprecated. Use generic option "-files" instead.
  -inputformat    <TextInputFormat(default)|SequenceFileAsTextInputFormat|JavaClassName>
                  Optional. The input format class.
  -outputformat   <TextOutputFormat(default)|JavaClassName>
                  Optional. The output format class.
  -partitioner    <JavaClassName>  Optional. The partitioner class.
  -numReduceTasks <n

In [11]:
%%bash
hdfs dfs -rm -r my_output

mapred streaming \
    -input hello.txt \
    -output my_output

Deleted my_output


2024-03-09 19:25:35,879 INFO Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
2024-03-09 19:25:39,177 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-03-09 19:25:39,347 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-03-09 19:25:39,347 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-03-09 19:25:39,389 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-03-09 19:25:39,668 INFO mapred.FileInputFormat: Total input files to process : 1
2024-03-09 19:25:39,702 INFO mapreduce.JobSubmitter: number of splits:1
2024-03-09 19:25:39,986 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local342617641_0001
2024-03-09 19:25:39,986 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-03-09 19:25:40,240 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2024-03-09 19:25:40,242 INFO mapreduce.Job: Runn

## Verify the result

In [12]:
%%bash

echo "Check if MapReduce job was successful"
hdfs dfs -test -e my_output/_SUCCESS
if [ $? -eq 0 ]; then
	echo "_SUCCESS exists!"
fi

Check if MapReduce job was successful
_SUCCESS exists!


Show output

In [13]:
!cat my_output/part-00000

0	Hello, World!


What happened here is that not having defined any mapper or reducer, the "Identity" mapper ([IdentityMapper](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/lib/IdentityMapper.html)) and reducer ([IdentityReducer](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/lib/IdentityReducer.html)) were used by default (see [Streaming command options](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Streaming_Command_Options)).

# Run a map-only MapReduce job

Not specifying mapper and reducer in the MapReduce job submission does not mean that MapReduce isn't going to run the mapper and reducer steps, it is simply going to use the Identity mapper and reducer.

To run a MapReduce job _without_ reducer one needs to use the generic option

    \-D mapreduce.job.reduces=0

(see [specifying map-only jobs](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Specifying_Map-Only_Jobs)).

In [14]:
%%bash
hdfs dfs -rm -r my_output

mapred streaming \
    -D mapreduce.job.reduces=0 \
    -input hello.txt \
    -output my_output

Deleted my_output


2024-03-09 19:25:45,234 INFO Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
2024-03-09 19:25:47,279 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-03-09 19:25:47,475 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-03-09 19:25:47,475 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-03-09 19:25:47,523 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-03-09 19:25:47,799 INFO mapred.FileInputFormat: Total input files to process : 1
2024-03-09 19:25:47,836 INFO mapreduce.JobSubmitter: number of splits:1
2024-03-09 19:25:48,169 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1258585303_0001
2024-03-09 19:25:48,169 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-03-09 19:25:48,434 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2024-03-09 19:25:48,435 INFO mapreduce.Job: Run

## Verify the result

In [15]:
!hdfs dfs -test -e my_output/_SUCCESS && cat my_output/part-00000

0	Hello, World!


The advantage of a map-only job is that the sorting and shuffling phases are skipped, so if you do not need that remember to specify `-D mapreduce.job.reduces=0 `.

On the other hand, a MapReduce job even with the default `IdentityReducer` will deliver sorted results because the data passed from the mapper to the reducer always gets sorted.

# Improved version of the MapReduce "Hello, World!" application

Taking into account the previous considerations, here's a more efficient version of the 'Hello, World!' application that bypasses the shuffling and sorting step.

In [16]:
%%bash
hdfs dfs -rm -r my_output

mapred streaming \
    -D mapreduce.job.reduces=0 \
    -input hello.txt \
    -output my_output \
    -mapper '/bin/cat'

Deleted my_output


2024-03-09 19:25:54,076 INFO Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
2024-03-09 19:25:56,212 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-03-09 19:25:56,427 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-03-09 19:25:56,427 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-03-09 19:25:56,465 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-03-09 19:25:56,762 INFO mapred.FileInputFormat: Total input files to process : 1
2024-03-09 19:25:56,797 INFO mapreduce.JobSubmitter: number of splits:1
2024-03-09 19:25:57,122 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local563996181_0001
2024-03-09 19:25:57,122 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-03-09 19:25:57,409 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2024-03-09 19:25:57,411 INFO mapreduce.Job: Runn

In [17]:
!hdfs dfs -test -e my_output/_SUCCESS && cat my_output/part-00000

Hello, World!	
